In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 198 kB 63.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=92b1557e83283a2e47f5244a91857872336fa4b22542b28907c92e40a4f78750
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import findspark
import datetime as dt

findspark.init()
from pyspark.sql import SparkSession
def init_spark(app_name: str):
 spark = SparkSession.builder.appName(app_name).getOrCreate()
 sc = spark.sparkContext
 return spark, sc
spark, sc = init_spark('demo')
sc

# Section 1 - Data Extraction
# Current file = queries.csv

# Transform the file to tsv format
import csv

with open('/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/queries.csv','r') as csvin,\
    open('/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/queries.tsv', 'w') as tsvout:
    csvin = csv.reader(csvin)
    tsvout = csv.writer(tsvout, delimiter='\t')
    for row in csvin:
        tsvout.writerow(row)

from pyspark.sql import SparkSession,Row, Column
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
queries_file = '/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/queries.tsv'
df = spark.read.csv(queries_file, header='True', inferSchema='True', sep='\t')


column_names = ["genres", "lang", "actors", "director", "cities", "country", 
                "from_realese_date", "production_company"]
# For all the above columns
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # Removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "'[]", "")\
                .alias(temp_name_1))\
  .drop(name)

  # Converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

print("Queries table:")
df.show()
queries_df = df

# ------------------------------------------------------------------------------

# Current file: credits.csv
import re
credits_file = '/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/credits.csv'
df = spark.read.csv(credits_file, header='True', inferSchema='True')

# load the data as you did before,
# just now change the delimiter to get evreything together
credits = spark.read.format("csv")\
.option("delimiter", "\t")\
.option("header","true")\
.option("inferSchema", "true")\
.load("/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/credits.csv")
prog = re.compile('\\[(.*?)\\]')
second_match = F.udf(lambda x: prog.findall(x)[1])
id_extract = F.udf(lambda x: x.split(",")[-1])
credits = credits\
.withColumn("id", id_extract("cast,crew,id"))\
.withColumn("cast", F.regexp_extract(F.col("cast,crew,id"), '\\[(.*?)\\]', 0
))\
.withColumn("crew", F.concat(F.lit("["),second_match("cast,crew,id"), F.lit(
"]")))\
.select("cast", "crew", "id")
df = credits
# df.printSchema()
column_names = ["cast", "crew"]
# For all the above columns
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # Removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "\\{\\[\\]'\\}", "")\
                .alias(temp_name_1))\
  .drop(name)

  # Converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

# For cast column - udf for extracting actors' names only from cast json string
actors_udf = F.udf(lambda arr: [arr[i][7:] for i in range(len(arr)) if i % 8 == 5])
df = df.withColumn('actors', actors_udf(F.col("cast")))\
  .drop("cast")

# For crew column - udf for extracting directors' names only from crew json string
directors_udf = F.udf(lambda arr: [arr[i+1][7:] for i in range(len(arr))
 if arr[i] == " job: Director"])
df = df.withColumn('directors', directors_udf(F.col("crew")))\
  .drop("crew")

# Converting arrays strings to arrays of strings
column_names = ["actors", "directors"]
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # Removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "\\{\\[\\]'\\}", "")\
                .alias(temp_name_1))\
  .drop(name)

  # Converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

print("Credits table:")
df.show()
credits_df = df

# ------------------------------------------------------------------------------

# Current file: movies.csv
movies_file = '/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/movies.csv'
df = spark.read.csv(movies_file, header='True', inferSchema='True')

# Doing the same process for all columns
column_names = ["genres", "production_companies", "production_countries", 
                "spoken_languages", "cities"]
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # Removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "\\{\\[\\]'\\}", "")\
                .alias(temp_name_1))\
  .drop(name)

  # Converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

# Finished working on cities column, and seperating production_companies
# because it has different structure
column_names = ["genres", "production_countries", "spoken_languages"]

# For each column - udf for extracting names only from json string
name_udf = F.udf(lambda arr: [arr[i][7:] for i in range(len(arr)) if i % 2 == 1])
for c in column_names:
  c_1 = c + "1"
  df = df.withColumn(c_1, name_udf(F.col(c)))\
    .drop(c)
prod_udf = F.udf(lambda arr: [arr[i][6:] for i in range(len(arr)) if i % 2 == 0])
df = df.withColumn("production_companies1", prod_udf(F.col("production_companies")))\
  .drop("production_companies")

# Renameing columns names
column_names = ["genres", "production_companies", "production_countries", 
                "spoken_languages"]
for name in column_names:
  current_name = name + "1"
  df = df.withColumnRenamed(current_name,name)

print("Movies table:")
df.show()
movies_df = df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Queries table:
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|user_id|              genres|                lang|              actors|            director|              cities|             country|from_realese_date|  production_company|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|    981| [Western,  Mystery]|  [English,  Srpski]|                  []|      [Nae Caranfil]|  [Haifa,  Tiberias]|                  []|           [2012]|[Katakuri-ke no K...|
|   3775|  [Action,  Western]|           [English]|                  []|                  []|          [Tel Aviv]|                  []|           [2013]|[Cl

In [ ]:
# Section 2 - Data Analysis

# Insight 1: Tel Aviv is the most queried city with 44721 lookups, Jerusalem is right after with 34561
# and then Haifa, Tiberias and Eilat with 22712, 9594 and 17348 lookups respectively
queries_itr = queries_df.rdd.toLocalIterator()
counters = {"Tel Aviv": 0, "Jerusalem": 0, "Haifa": 0, "Tiberias": 0, "Eilat": 0,
          " Tel Aviv": 0, " Jerusalem": 0, " Haifa": 0, " Tiberias": 0, " Eilat": 0}
for row in queries_itr:
  for city in row.cities:
    counters[city] += 1

counters_final = {"Tel Aviv": counters["Tel Aviv"] + counters[" Tel Aviv"],
                "Jerusalem": counters["Jerusalem"] + counters[" Jerusalem"],
                "Haifa": counters["Haifa"] + counters[" Haifa"],
                "Tiberias": counters["Tiberias"] + counters[" Tiberias"],
                "Eilat": counters["Eilat"] + counters[" Eilat"]}

print("Insight 1: Tel Aviv is the most queried city with 44721 lookups, Jerusalem is right after with 34561, and then Haifa, Tiberias and Eilat with 22712, 9594 and 17348 lookups respectively")
print(counters_final)
print()

# Insight 2: Jerusalemites and Tel-Avivians are almost the only ones to search for movies in Hebrew
queries_itr = queries_df.rdd.toLocalIterator()
counters = {"Tel Aviv": 0, "Jerusalem": 0, "Haifa": 0, "Tiberias": 0, "Eilat": 0,
            " Tel Aviv": 0, " Jerusalem": 0, " Haifa": 0, " Tiberias": 0, " Eilat": 0}

for row in queries_itr:
  if " עִבְרִית" in row.lang:
    for city in row.cities:
      counters[city] += 1

counters_final = {"Tel Aviv": counters["Tel Aviv"] + counters[" Tel Aviv"],
                  "Jerusalem": counters["Jerusalem"] + counters[" Jerusalem"],
                  "Haifa": counters["Haifa"] + counters[" Haifa"],
                  "Tiberias": counters["Tiberias"] + counters[" Tiberias"],
                  "Eilat": counters["Eilat"] + counters[" Eilat"]}

print("Insight 2: Jerusalemites and Tel-Avivians are almost the only ones to search for movies in Hebrew:")
print(counters_final)
print()


# Insight 3: Jerusalemites care about the playing actors, Haifa people care 
# about directors, and Eilaties alaways specify countries in their queries.
print("Insight 3: Jerusalemites care about the playing actors, Haifa people care about directors, and Eilaties alaways specify countries in their queries.")
queries_itr = queries_df.rdd.toLocalIterator()
counters = {"Tel Aviv": 0, "Jerusalem": 0, "Haifa": 0, "Tiberias": 0, "Eilat": 0,
          " Tel Aviv": 0, " Jerusalem": 0, " Haifa": 0, " Tiberias": 0, " Eilat": 0}
for row in queries_itr:
  if row.actors[0] == "":
    for city in row.cities:
      counters[city] += 1

counters_final = {"Tel Aviv": counters["Tel Aviv"] + counters[" Tel Aviv"],
                "Jerusalem": counters["Jerusalem"] + counters[" Jerusalem"],
                "Haifa": counters["Haifa"] + counters[" Haifa"],
                "Tiberias": counters["Tiberias"] + counters[" Tiberias"],
                "Eilat": counters["Eilat"] + counters[" Eilat"]}

print()
print("actors empty by city:")
print(counters_final)
print()

queries_itr = queries_df.rdd.toLocalIterator()
counters = {"Tel Aviv": 0, "Jerusalem": 0, "Haifa": 0, "Tiberias": 0, "Eilat": 0,
          " Tel Aviv": 0, " Jerusalem": 0, " Haifa": 0, " Tiberias": 0, " Eilat": 0}
for row in queries_itr:
  if row.director[0] == "":
    for city in row.cities:
      counters[city] += 1

counters_final = {"Tel Aviv": counters["Tel Aviv"] + counters[" Tel Aviv"],
                "Jerusalem": counters["Jerusalem"] + counters[" Jerusalem"],
                "Haifa": counters["Haifa"] + counters[" Haifa"],
                "Tiberias": counters["Tiberias"] + counters[" Tiberias"],
                "Eilat": counters["Eilat"] + counters[" Eilat"]}

print("director empty by city:")
print(counters_final)
print()

queries_itr = queries_df.rdd.toLocalIterator()
counters = {"Tel Aviv": 0, "Jerusalem": 0, "Haifa": 0, "Tiberias": 0, "Eilat": 0,
          " Tel Aviv": 0, " Jerusalem": 0, " Haifa": 0, " Tiberias": 0, " Eilat": 0}
for row in queries_itr:
  if row.country[0] == "":
    for city in row.cities:
      counters[city] += 1

counters_final = {"Tel Aviv": counters["Tel Aviv"] + counters[" Tel Aviv"],
                "Jerusalem": counters["Jerusalem"] + counters[" Jerusalem"],
                "Haifa": counters["Haifa"] + counters[" Haifa"],
                "Tiberias": counters["Tiberias"] + counters[" Tiberias"],
                "Eilat": counters["Eilat"] + counters[" Eilat"]}

print("country empty by city:")
print(counters_final)
print()

# Insight 4: people who look for movies in Eilat, never look for movies in over places in the same query
queries_itr = queries_df.rdd.toLocalIterator()
city_lookup_dict = {}
for row in queries_itr:
  key = " ".join(str(x) for x in row.cities)
  if key not in city_lookup_dict.keys():
    city_lookup_dict[key] = 1
  else:
    city_lookup_dict[key] += 1
print("Insight 4: people who look for movies in Eilat, never look for movies in over places in the same query")
print("Lookups for cinemas in different cities counts")
print(city_lookup_dict)
print()

# Insight 5: the number of tickets in reservations are more or less in the same amount and from the same distribution
print("Insight 5: the number of tickets in reservations are more or less in the same amount and from the same distribution")
import pandas as pd
tickets_by_city = [["Tel Aviv", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ["Jerusalem", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   ["Haifa", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ["Tiberias", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   ["Eilat", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
tickets_by_city = pd.DataFrame(tickets_by_city, columns=["City", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
tickets_by_city.set_index("City", inplace=True)

tickets_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/tickets.csv',
                            header='True', inferSchema='True')
tickets_itr = tickets_df.rdd.toLocalIterator()
for row in tickets_itr:
  tickets_by_city.at[row.city, str(row.number_of_tickets)] += 1

print("Number of tickets per reservation count by city")
print(tickets_by_city)
print()
ax = tickets_by_city.plot.bar(rot=0)
ax.plot()

# Insight 6: Approximately 95% of the users looked for movies in their home towns
print("Insight 6: Approximately 95% of the users looked for movies in their home towns")
users_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/users.csv',
                          header='True', inferSchema='True')
unit = users_df.join(queries_df, on=users_df.user_id == queries_df.user_id)
unit_itr = unit.rdd.toLocalIterator()
cnt = 0
for row in unit_itr:
  loc = row.user_location
  loc_ = " " + row.user_location
  if loc in row.cities or loc_ in row.cities:
    cnt += 1

print(cnt/100000)
print()

# Insight 7: Most popular geners, actors, directors, languages, languages, countries,
# production companies by city
print("Insight 7: Most popular geners, actors, directors, languages, languages, countries, production companies by city")
queries_itr = queries_df.rdd.toLocalIterator()

all_genres = set()
all_actors = set()
all_directors = set()
all_languages = set()
all_countries = set()
all_prod_companies = set()
all_years = []

queries_itr = queries_df.rdd.toLocalIterator()
for row in queries_itr:
  for g in row.genres:
    if g != "" and not(g.lstrip() in all_genres):
      all_genres.add(g.lstrip())
  for a in row.actors:
    if a != "" and not(a.lstrip() in all_actors):
      all_actors.add(a.lstrip())
  for d in row.director:
    if d != "" and not(d.lstrip() in all_directors):
      all_directors.add(d.lstrip())
  for l in row.lang:
    if l != "" and not(l.lstrip() in all_languages):
      all_languages.add(l.lstrip())
  for c in row.country:
    if c != "" and not(c.lstrip() in all_countries):
      all_countries.add(c.lstrip())
  for c in row.production_company:
    if c != "" and not(c.lstrip() in all_prod_companies):
      all_prod_companies.add(c.lstrip())
  all_years.append(int(row.from_realese_date[0]))

all_dict = {"genres": all_genres, "actors": all_actors, "director":all_directors, 
        "lang": all_languages, "country": all_countries, 
        "production_company": all_prod_companies}

for city in ["Tel Aviv", "Jerusalem", "Haifa", "Eilat", "Tiberias"]:
  print(city)
  for name, all_something in zip(all_dict.keys(), all_dict.values()):
    print(name)
    queries_itr = queries_df.rdd.toLocalIterator()
    
    counters = {}
    for s in all_something:
      counters[s] = 0

    for row in queries_itr:
      for s in row[name]:
        city2 = " " + city
        if s != "" and ( city in row.cities or city2 in row.cities ):
          counters[s.lstrip()] += 1

    counters = sorted(counters.items(), key=lambda item: item[1], reverse=True)

    print(counters)
  print()


# Insight 8: the minimal queried year is 1990
print("Insight 8: the minimal queried year is 1990")
print(f"min={min(all_years)}, max={max(all_years)}, avg={sum(all_years) / len(all_years)}")

Output hidden; open in https://colab.research.google.com to view.

## Explanations for the data analysis section

In this part of the project we have chosen to focus on finding insights mainly from the "queries" table as well as from the "tickets" table, and in particular those that will help us in the horizontal and vertical fregmentation.

Since the general task in this part of the project is to divide the database between sites that are geographically close to the cities of Haifa, Tiberias, Tel Aviv, Jerusalem and Eilat, we found it appropriate to focus on the differences embodied in the database between the cities, hoping that if we find significant differences between cities, we would be able to generalize them for certain rows / columns in the database.

Indeed, we have found some interesting insights, as one can see from the above code output:

1. Tel Aviv is the most queried city with 44721 lookups, Jerusalem is right after with 34561 and then Haifa, Tiberias and Eilat with 22712, 9594 and 17348 lookups respectively
2. Jerusalemites and Tel-Avivians are almost the only ones to search for movies in Hebrew.
4. People who look for movies in Eilat, never look for movies in over places in the same query.
5. The number of tickets in reservations are more or less in the same amount and from the same distribution.
6. Approximately 95% of the users looked for movies in their hometowns.
7. Popular features in Tel Aviv:
   * Genre: Action
   * Actors: Tom Hanks, Jonny Depp, Brad Pitt
   * Languages: English, Hebrew
   * Country: Israel
8. Popular features in Jerusalem:
   * Actors: Tom Hanks, Jonny Depp, Brad Pitt
   * Languages: English, Hebrew
   * Country: Israel
9. Popular features in Haifa:
   * Genre: Drama
   * Languages: English
10. Popular features in Eilat:
   * Genre: All are similarly popular
   * Languages: English
   * Production comanies: Pixar Animation Studios, Walt Disney Pictures, Warner Bros
11. Popular features in Tiberias:
   * Ggenre: Drama, Documentary, Family
   * Actors: Tom Hanks, Jonny Depp, Tom Hanks
   * Languages: English
12. The oldest queried movie is from 1990, even thought there are many older movies in the movies table.

